<a href="https://colab.research.google.com/github/taniokah/Sports_Analysis/blob/master/Player_Tracking_for_ICCP2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%matplotlib inline
import cv2 #　opencvのインポート
import matplotlib.pyplot as plt # matplotlib(描画用)

print(cv2.__version__)

3.4.3


In [4]:
!pip install pytube
!pip install ffmpeg

In [0]:
!rm -rf screen_caps
!mkdir screen_caps

In [0]:
from pytube import YouTube

yt = YouTube("https://youtu.be/l00CwQ6HI_o")


In [7]:
yt.streams.all()
#yt.streams.filter(progressive=False).desc().all()
#yt.streams.get_by_itag(313)
#yt.streams.first().download()
yt.streams.get_by_itag(313).download()

'/content/20190224 鹿屋体育大学 vs 鹿児島高専 1st.webm'

In [0]:
import cv2
import numpy as np

#動画ファイルを読み込む
#file_name = u"20190217 徳島大学医学部 vs 吉野クラブ 2nd No2.mp4"
#file_name = u"20190224 鹿屋体育大学 vs 鹿児島高専 1st.mp4"
file_name = u"/content/20190224 鹿屋体育大学 vs 鹿児島高専 1st.webm"
cap = cv2.VideoCapture(file_name)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(str(width) + ', ' + str(height))
fps = cap.get(cv2.CAP_PROP_FPS)

#(width, height) = (1280, 720)
print(str(width) + ', ' + str(height))
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
writer = cv2.VideoWriter('T20190224_1st.mp4', fourcc, fps, (width, height), True)

#count = 0

while True:
	#_, frame = cap.read()
	ret, frame = cap.read()
	if not ret:
		break

	pts1 = np.float32([[1110, 495], [2700, 495], [-90, 1065], [3870, 1065]])
	pts2 = np.float32([[0, 0], [width, 0], [0, height], [width, height]])
	matrix = cv2.getPerspectiveTransform(pts1, pts2)
	result = cv2.warpPerspective(frame, matrix, (width, height))

	#cv2.circle(frame, (370, 165), 5, (0, 0, 255), -1)
	#cv2.circle(frame, (900, 165), 5, (0, 255, 0), -1)
	#cv2.circle(frame, (-30, 355), 5, (255, 0, 0), -1)
	#cv2.circle(frame, (1290, 355), 5, (0, 255, 255), -1)

	#cv2_imshow(frame)
	#cv2.imshow("Perspective transformation", result)

	writer.write(result)
  
	key = cv2.waitKey(1)
	if key == 27:
		break
  
#	count = count + 1
#	if count > 1000:
#		break

cv2.destroyAllWindows()
writer.release()
cap.release()
print('done')


3840, 2160
3840, 2160
